
# Aspect Classification Experiments
Classify feedback into aspects (e.g., teaching skills, behaviour, knowledge, relevancy, general) with interpretable and advanced methods plus generalization checks.


In [ ]:

import pandas as pd
from pathlib import Path

DATA_PATH = Path('data_feedback.xlsx')
df = pd.read_excel(DATA_PATH)
print(df.head())


In [ ]:

print('Aspect label distribution:')
print(df['aspect'].value_counts())
print('
Teacher/course breakdown per aspect:')
print(pd.crosstab(df['aspect'], df['teacher/course']))



## 1. Glossary + promptable definitions
Helpful for explainability and for guiding annotators/LLM prompts.


In [ ]:

ASPECT_DEFINITIONS = {
    'teaching skills': 'Pedagogy, clarity, preparedness, delivery quality.',
    'behaviour': 'Politeness, supportiveness, attitude toward students.',
    'knowledge': 'Subject-matter expertise and depth.',
    'relevancy': 'Alignment of course content with needs or curriculum.',
    'general': 'Generic praise/critique not tied to a specific trait.',
}
for k,v in ASPECT_DEFINITIONS.items():
    print(f"{k}: {v}")



## 2. Baseline: word TF–IDF + Logistic Regression


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

X_train, X_val, y_train, y_val = train_test_split(
    df['comments'], df['aspect'], test_size=0.3, random_state=42, stratify=df['aspect']
)

word_lr = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2), min_df=1)),
    ('clf', LogisticRegression(max_iter=300, class_weight='balanced', multi_class='auto'))
])
word_lr.fit(X_train, y_train)
preds = word_lr.predict(X_val)
print(classification_report(y_val, preds))
ConfusionMatrixDisplay.from_predictions(y_val, preds, normalize='true', cmap='Greens')
plt.title('Aspect TF–IDF baseline')
plt.show()



### SHAP explanations for aspect model


In [ ]:

import shap

explainer = shap.LinearExplainer(word_lr.named_steps['clf'], word_lr.named_steps['tfidf'].transform(X_train))
val_tfidf = word_lr.named_steps['tfidf'].transform(X_val)
shap_values = explainer(val_tfidf)
shap.plots.bar(shap_values)



## 3. Character n-grams baseline


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer as CharTfidf

char_lr = Pipeline([
    ('tfidf', CharTfidf(analyzer='char', ngram_range=(3,5), min_df=1)),
    ('clf', LogisticRegression(max_iter=300, class_weight='balanced', multi_class='auto'))
])
char_lr.fit(X_train, y_train)
char_preds = char_lr.predict(X_val)
print(classification_report(y_val, char_preds))
ConfusionMatrixDisplay.from_predictions(y_val, char_preds, normalize='true', cmap='Oranges')
plt.title('Aspect character baseline')
plt.show()



## 4. Hierarchical idea: detect domain then aspect
Train a coarse teacher/course classifier first, then specialized aspect models per domain to check gains.


In [ ]:

from sklearn.preprocessing import LabelEncoder

coarse = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
    ('clf', LogisticRegression(max_iter=200))
])
coarse.fit(df['comments'], df['teacher/course'])

teacher_mask = df['teacher/course'].str.lower().eq('teacher')
teacher_model = word_lr.fit(df.loc[teacher_mask, 'comments'], df.loc[teacher_mask, 'aspect'])
course_model = word_lr.fit(df.loc[~teacher_mask, 'comments'], df.loc[~teacher_mask, 'aspect'])

def hierarchical_predict(text):
    domain = coarse.predict([text])[0]
    if domain.lower() == 'teacher':
        return teacher_model.predict([text])[0]
    return course_model.predict([text])[0]

print('Hierarchical prediction example:', hierarchical_predict(df['comments'].iloc[0]))



## 5. Cross-domain holdout
Train on teacher comments, test on course comments (and vice versa) to measure generalization.


In [ ]:

from sklearn.metrics import accuracy_score

teacher_mask = df['teacher/course'].str.lower().eq('teacher')
course_mask = df['teacher/course'].str.lower().eq('course')

model = word_lr
model.fit(df.loc[teacher_mask, 'comments'], df.loc[teacher_mask, 'aspect'])
acc_teacher_to_course = accuracy_score(df.loc[course_mask, 'aspect'], model.predict(df.loc[course_mask, 'comments']))

model.fit(df.loc[course_mask, 'comments'], df.loc[course_mask, 'aspect'])
acc_course_to_teacher = accuracy_score(df.loc[teacher_mask, 'aspect'], model.predict(df.loc[teacher_mask, 'comments']))

print({'teacher→course': acc_teacher_to_course, 'course→teacher': acc_course_to_teacher})



## 6. Transformer fine-tuning


In [ ]:

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np

model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
label_list = sorted(df['aspect'].unique())
label_to_id = {l:i for i,l in enumerate(label_list)}
id_to_label = {i:l for l,i in label_to_id.items()}

train_ds = Dataset.from_pandas(pd.DataFrame({'text': X_train, 'label': y_train.map(label_to_id)}))
val_ds = Dataset.from_pandas(pd.DataFrame({'text': X_val, 'label': y_val.map(label_to_id)}))

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)
train_ds = train_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_list))
args = TrainingArguments(
    output_dir='aspect-model',
    evaluation_strategy='epoch',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-5,
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'macro_f1': f1, 'precision': precision, 'recall': recall}

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
# trainer.train()



### Zero-shot aspect probing


In [ ]:

from transformers import pipeline

zs = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')
labels = list(ASPECT_DEFINITIONS.keys())
example_text = df['comments'].iloc[1]
print(zs(example_text, candidate_labels=labels, hypothesis_template='This feedback is about {label}.'))



## 7. Error analysis and ethics notes


In [ ]:

errors = pd.DataFrame({'text': X_val, 'true': y_val, 'pred': preds})
errors = errors[errors['true'] != errors['pred']]
errors['length'] = errors['text'].str.len()
errors['domain'] = errors['text'].apply(lambda t: 'teacher' if 'teacher' in t.lower() else 'course' if 'course' in t.lower() else 'unknown')
print(errors)
